In [12]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModel
import torch
import subprocess
from pprint import pprint

## 讀檔案

In [2]:
df = pd.read_csv(r"C:\Users\user\Desktop\期末專題\爬蟲\104_職缺\Crawler_1140718\104_programmers_jobs.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15748 entries, 0 to 15747
Data columns (total 46 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   job_id  15748 non-null  object 
 1   更新日期    15748 non-null  object 
 2   查詢職類    15748 non-null  int64  
 3   查詢關鍵字   15748 non-null  object 
 4   職務類別    15748 non-null  object 
 5   職缺名稱    15748 non-null  object 
 6   公司名稱    15748 non-null  object 
 7   公司連結    15748 non-null  object 
 8   產業別     15748 non-null  object 
 9   上班地點    15748 non-null  object 
 10  地區      14577 non-null  object 
 11  城市      15748 non-null  object 
 12  國家      15748 non-null  object 
 13  薪資      15748 non-null  object 
 14  薪資下限    15748 non-null  int64  
 15  薪資上限    15748 non-null  int64  
 16  職缺描述    15733 non-null  object 
 17  職務需求    0 non-null      float64
 18  工作經歷要求  15748 non-null  object 
 19  學歷要求    15748 non-null  object 
 20  科系要求    8534 non-null   object 
 21  擅長工具    11180 non-null  object 
 22

In [26]:
df = df.head(5).copy()

df["技能原文"] = (
    df["職缺描述"].astype(str) + "\n" +
    df["擅長工具"].astype(str) + "\n" +
    df["工作技能"].astype(str)
)

In [42]:
def query_ollama(prompt):
    result = subprocess.run(
        ["ollama", "run", "qwen:7b"],
        input=prompt.encode(),
        stdout=subprocess.PIPE
    )
    return result.stdout.decode()

def extract_skills(text):
    prompt = f"""

    請從以下內容中提取技能，列出技能或工具，用逗號分隔，
    內容中不要有其他文字
    (EX：不要有:以下是从提供的内容中提取的技能...等文字)
    ，單純回應技能名稱(如果有中英文版本給我中英文版本)
    (EX：HTML、CSS...等。)
    
    ：{text}

    """
    
    return query_ollama(prompt).strip()

# 新增提取技能欄位




In [43]:
df["技能"] = df["技能原文"].apply(extract_skills)
df.to_csv("output_with_skills.csv", index=False)

df_test = df[["技能", "技能原文"]]
df_test

,技能,技能原文
0,前端开发技能：HTML、CSS、JavaScript、React、Next.js、Vue、N...,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...
1,前端开发技能：\n- HTML\n- CSS\n- JavaScript\n- TypeSc...,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...
2,技能名称：\n\n1. 英文阅读撰寫能力（中级以上）\n2. Backend程式开发经验 (...,工作內容 - 負責 Backend 程式設計並擬定開發計畫 - 檢視並優化服務安全性穩定性...
3,技能列表：\n\n1. Java\n2. PHP\n3. Go\n4. JavaScript...,"Yating(https://yating.tw/zh-hant), a Taiwan AI..."
4,技能列表如下（以逗号分隔）：\n\n- HTML\n- CSS\n- JavaScript\...,主要職責： • 設計和實施高效能的前端架構。 • 領導前端項目，從規劃到部署，主要使用 Vu...


In [53]:
print(df_test["技能"].iloc[0])

前端开发技能：HTML、CSS、JavaScript、React、Next.js、Vue、Nuxt.js

WebXR相关技能：Three.js、TypeScript、WebGL、沉浸式体验开发经验

其他工具或语言：Git、Redux、Node.js、AWS、Github等。熟悉结构化程式设计和系统整合分析。
